### Imports
Please make sure to run this first to set up the environment. This can take around a few minutes

In [ ]:
#You do not need to make any changes to this cell, only run the cell by using the button on the left

%%capture
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy.linalg import norm
model = SentenceTransformer("avsolatorio/GIST-large-Embedding-v0", revision=None)


!pip install pymupdf
!pip install -q -U google-generativeai
!pip install langchain-google-genai
!pip install ragas


from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, TokenTextSplitter
import google.generativeai as genai

In [ ]:
#You do not need to make any changes to this cell, only run the cell by using the button on the left

def cos(A, B, normalize=True): #normalize for cos sim, or do dot product
  """
  returns similarity between 2 vecors using cos sim
  """
  A, B = A, B
  if normalize:
      return (A @ B.T)/(norm(A)*norm(B))
  return A @ B.T

def read_file(file):
  """
  read in pdf or txt file
  """
  if file[-3:] == "txt":
    f = open(file)
    return f.read()
  elif file[-3:] == "pdf":
    import pymupdf
    text = ""
    doc = pymupdf.open(file)
    for page in doc:
      text += page.get_text()
    return text

  else:
    print("Please use a pdf or txt file for this task")
    return None


def generate(model, GOOGLE_API_KEY, query):
    """
    Creates a call to the API
    Takes in a model to choose which API
    Returns the updated tokens (int, int) and API response (str)
    """
    if model[0:6] == "gemini":
        genai.configure(api_key=GOOGLE_API_KEY)
        model = genai.GenerativeModel(model)
        response = model.generate_content(query)
        return response.text

    else: raise (f"Error, model {model} not found")

In [ ]:
config = {
    "model": "gemini-2.0-flash",  # or other model IDs
    "temperature": 0.4,
    "max_tokens": None,
    "top_p": 0.8,
}

### 1. Upload a document of your choosing. Try and keep it a reasonable length.

\* To upload files, click the folder icon on the left hand side of the screen and select "Upload to session storage".This will only keep the file as long as the session is running so if you come back to this task later, you will need to reupload your files *

In [ ]:
# Insert your path here
file_path = '/content/tiger sharks.pdf' # From the left window, you can right click to copy the path

document = read_file(file_path)

In [ ]:
#You do not need to make any changes to this cell, only run the cell by using the button on the left

print(document) # Make sure everything looks right

Tiger shark
Temporal range:
In The Bahamas
Conservation status
Near Threatened  (IUCN 3.1)[1]
Apparently Secure  (NatureServe)[2]
Scientific classification
Domain:
Eukaryota
Kingdom:
Animalia
Phylum:
Chordata
Class:
Chondrichthyes
Subclass:
Elasmobranchii
Division:
Selachii
Order:
Carcharhiniformes
Tiger shark
The tiger shark (Galeocerdo cuvier)[3] is a species of
ground shark, and the only extant member of the genus
Galeocerdo and family Galeocerdonidae. It is a large apex
predator, with females capable of attaining a length of over
5 m (16 ft 5 in).[4] Populations are found in many tropical and
temperate waters, especially around central Pacific islands. Its
name derives from the dark stripes down its body, which
resemble a tiger's pattern, but fade as the shark matures.[5]
The tiger shark is a solitary, mostly nocturnal hunter. It is
notable for having the widest food spectrum of all sharks, with
a range of prey that includes crustaceans, fish, seals, birds,
squid, turtles, sea snak

### 2. Chunk your document and create local database

In [ ]:
#You do not need to make any changes to this cell, only run the cell by using the button on the left

def get_text_splitter(splitter_type="character", chunk_size=1000, chunk_overlap=200):
    """
    Args:
        splitter_type (str): The type of text splitter to use ('character', 'recursive', 'token').
        chunk_size (int): The maximum size of each text chunk.
        chunk_overlap (int): The overlap between text chunks.
    """
    if splitter_type == "character":
        return CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    elif splitter_type == "recursive":
        return RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    elif splitter_type == "token":
        return TokenTextSplitter(chunk_size=chunk_size)
    else:
        raise ValueError("Invalid splitter type. Choose from 'character', 'recursive', or 'token'.")


def chunk_data(doc, splitter, chunk_size=1000, overlap=50):
  splitter = get_text_splitter(splitter, chunk_size=chunk_size, chunk_overlap=overlap)
  doc_chunks = splitter.split_text(doc)
  documents = splitter.create_documents(doc_chunks)
  return documents

documents = [i.page_content for i in chunk_data(document, "recursive", chunk_size=1000, overlap=50)]

In [ ]:
#You do not need to make any changes to this cell, only run the cell by using the button on the left

# Ensure chunks look correct
print(f"Split into {len(documents)} chunks")
print("Chunk 1: \n", documents[0])
print(type(documents))

Split into 24 chunks
Chunk 1: 
 Tiger shark
Temporal range:
In The Bahamas
Conservation status
Near Threatened  (IUCN 3.1)[1]
Apparently Secure  (NatureServe)[2]
Scientific classification
Domain:
Eukaryota
Kingdom:
Animalia
Phylum:
Chordata
Class:
Chondrichthyes
Subclass:
Elasmobranchii
Division:
Selachii
Order:
Carcharhiniformes
Tiger shark
The tiger shark (Galeocerdo cuvier)[3] is a species of
ground shark, and the only extant member of the genus
Galeocerdo and family Galeocerdonidae. It is a large apex
predator, with females capable of attaining a length of over
5 m (16 ft 5 in).[4] Populations are found in many tropical and
temperate waters, especially around central Pacific islands. Its
name derives from the dark stripes down its body, which
resemble a tiger's pattern, but fade as the shark matures.[5]
The tiger shark is a solitary, mostly nocturnal hunter. It is
notable for having the widest food spectrum of all sharks, with
a range of prey that includes crustaceans, fish, seals,

In [ ]:
#You do not need to make any changes to this cell, only run the cell by using the button on the left

#Create local embeddings database
#This may take some time if you have many chunks
embeddings = np.stack([model.encode(i) for i in documents])
print(embeddings.shape) # (num chunks, embed dim)

(24, 1024)


### 3. Simple RAG setup

This function implements the most basic version of RAG. You are welcome to modify the prompt, but it is not required. The function returns both the LLM output as well as the retieved context for use in evaluation.

In [ ]:
#You do not need to make any changes to this cell, only run the cell by using the button on the left

def RAG(query, database, embeddings, k=1):
  import torch

  query_vector = model.encode(query)

  context = []
  for c in torch.topk(torch.from_numpy(cos(query_vector, embeddings)), k)[1]:
    context.append(database[c])

  prompt = f'''
  You will be given a user query and context. Use the context given to
  you to ground your response to answer the user query

  USER QUERY: {query}
  CONTEXT: {"".join(context)}
  '''
                             #gemini-2.0-flash-lite#
  generation = generate(model="gemini-2.0-flash", GOOGLE_API_KEY=os.environ["GOOGLE_API_KEY"], query=prompt)

  return context, generation

## TASK: Using RAGAS to evaluate outputs

Before we can evaluate any outputs, RAGAS requires a small amount of setup. First, ensure you gemini API key is connected:

In [ ]:
#You do not need to make any changes to this cell, only run the cell by using the button on the left

from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
#You do not need to make any changes to this cell, only run the cell by using the button on the left

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_google_genai import ChatGoogleGenerativeAI


# Initialize evaluator with Google AI Studio
evaluator_llm = LangchainLLMWrapper(ChatGoogleGenerativeAI(
    model=config["model"],
    temperature=config["temperature"],
    max_tokens=config["max_tokens"],
    top_p=config["top_p"],
))

from langchain_google_genai import GoogleGenerativeAIEmbeddings
evaluator_embeddings = LangchainEmbeddingsWrapper(GoogleGenerativeAIEmbeddings(
    model="models/embedding-004",  # Google's text embedding model
    task_type="retrieval_document"  # Optional: specify the task type
))

### We are now ready to evaluate some outputs!






In [ ]:
# Insert your query and your reference here and then run the cells below to check each metric
# Repeat for the following 4 unique queries for your database
query = "Do Tiger Sharks attack humans?"
reference="Tiger sharks are known to be one of the most dangerous shark species to humans and ranks second on recorded sharks attacks on humans."
context, generation = RAG(query, documents, embeddings, k=5)
print(context)
print(generation)

['people swim, surf, and dive in Hawaiian waters every day.[63] Human interactions with tiger sharks in\nHawaiian waters have been shown to increase between September and November, when tiger shark\nfemales are believed to migrate to the islands to give birth.[64]\nOn 8 June 2023, a tiger shark attacked and killed a 23-year-old Russian man in the Red Sea off the\ncoast of the Egyptian city of Hurghada. The attack was filmed by onlookers and the recording went\nviral. The shark was later captured by fishermen and killed. This was the third fatal tiger shark attack\nin the area since 2022.[65]\nBetween 1959 and 1976, 4,668 tiger sharks were culled in the state of Hawaii in an effort to protect the\ntourism industry.[66] Despite damaging the shark population, these efforts were shown to be\nineffective in decreasing the number of interactions between humans and tiger sharks. Feeding sharks\nin Hawaii (except for traditional Hawaiian cultural or religious practices) is illegal,[67][68] and

In [ ]:
#IMPLEMENTATION OF Context Precision
from ragas import SingleTurnSample
from ragas.metrics import LLMContextPrecisionWithoutReference

context_precision = LLMContextPrecisionWithoutReference(llm=evaluator_llm)

sample = SingleTurnSample(
    user_input=query,
    response=generation,
    retrieved_contexts=context,
)
await context_precision.single_turn_ascore(sample)

0.9166666666361111

In [ ]:
#IMPLEMENTATION OF Context Recall
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import LLMContextRecall

sample = SingleTurnSample(
    user_input=query,
    response=generation,
    reference=reference,
    retrieved_contexts=context,
)

context_recall = LLMContextRecall(llm=evaluator_llm)
await context_recall.single_turn_ascore(sample)

1.0

In [ ]:
#IMPLEMENTATION OF Content Entities Recall
from ragas import SingleTurnSample
from ragas.metrics import ContextEntityRecall

sample = SingleTurnSample(
    reference=generation,
    retrieved_contexts=context,
)

scorer = ContextEntityRecall(llm=evaluator_llm)

await scorer.single_turn_ascore(sample)

0.4999999975

In [ ]:
#IMPLEMENTATION OF Noise Sensitivity
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import NoiseSensitivity

sample = SingleTurnSample(
    user_input=query,
    response=generation,
    reference=reference,
    retrieved_contexts=context
)

scorer = NoiseSensitivity(llm=evaluator_llm)
await scorer.single_turn_ascore(sample)

np.float64(0.6)

In [ ]:
#IMPLEMENTATION OF Faithfulness
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import Faithfulness

sample = SingleTurnSample(
        user_input=query,
        response=generation,
        retrieved_contexts=context
    )
scorer = Faithfulness(llm=evaluator_llm)
await scorer.single_turn_ascore(sample)

0.9